In [54]:
import pandas as pd
import yfinance as yf
import yahoo_fin.stock_info as si
from yahoo_fin.stock_info import get_data

1. Exploration

We can start by checking how much tickers there are in the NASDAQ stocks market:

In [55]:
nas_list=si.tickers_nasdaq()
print("Tickers in Nasdaq:", len(nas_list))
print(nas_list[0:15])
nasdaq_list=nas_list[0:15]

Tickers in Nasdaq: 4798
['AACG', 'AADI', 'AADR', 'AAL', 'AAME', 'AAOI', 'AAON', 'AAPB', 'AAPD', 'AAPL', 'AAPU', 'AAXJ', 'ABAT', 'ABCL', 'ABCS']


We can take as an example APPLE:

In [56]:
nas_aapl= get_data("aapl", start_date="11/30/2019", end_date="11/30/2024", index_as_date = True, interval="1d") #différents interval (1m to 3months)
nas_aapl

,open,high,low,close,adjclose,volume,ticker
2019-12-02,66.817497,67.062500,65.862503,66.040001,64.024628,94487200,AAPL
2019-12-03,64.577499,64.882500,64.072502,64.862503,62.883068,114430400,AAPL
2019-12-04,65.267502,65.827499,65.169998,65.434998,63.438103,67181600,AAPL
2019-12-05,65.947502,66.472504,65.682503,66.394997,64.368797,74424400,AAPL
2019-12-06,66.870003,67.750000,66.824997,67.677498,65.612144,106075600,AAPL
...,...,...,...,...,...,...,...
2024-11-22,228.059998,230.720001,228.059998,229.869995,229.869995,38168300,AAPL
2024-11-25,231.460007,233.250000,229.740005,232.869995,232.869995,90152800,AAPL
2024-11-26,233.330002,235.570007,233.330002,235.059998,235.059998,45986200,AAPL
2024-11-27,234.470001,235.690002,233.809998,234.929993,234.929993,33498400,AAPL


Informations about the data:

In [57]:
print(nas_aapl.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1258 entries, 2019-12-02 to 2024-11-29
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   open      1258 non-null   float64
 1   high      1258 non-null   float64
 2   low       1258 non-null   float64
 3   close     1258 non-null   float64
 4   adjclose  1258 non-null   float64
 5   volume    1258 non-null   int64  
 6   ticker    1258 non-null   object 
dtypes: float64(5), int64(1), object(1)
memory usage: 78.6+ KB
None


In [58]:
nas_aapl.head()

,open,high,low,close,adjclose,volume,ticker
2019-12-02,66.817497,67.062500,65.862503,66.040001,64.024628,94487200,AAPL
2019-12-03,64.577499,64.882500,64.072502,64.862503,62.883068,114430400,AAPL
2019-12-04,65.267502,65.827499,65.169998,65.434998,63.438103,67181600,AAPL
2019-12-05,65.947502,66.472504,65.682503,66.394997,64.368797,74424400,AAPL
2019-12-06,66.870003,67.750000,66.824997,67.677498,65.612144,106075600,AAPL


We put all the data about the charts in a dictionnary:

In [59]:
ticker_list = ["amzn", "aapl", "ba"]
dict_data_tickers = {}
for ticker in nasdaq_list:
    dict_data_tickers[ticker] = get_data(ticker, start_date="11/30/2014", end_date="11/30/2024", index_as_date = True, interval="1d")

Example of how to access it:

In [60]:
print(dict_data_tickers[list(dict_data_tickers)[0]])

            open  high   low  close  adjclose  volume ticker
2014-12-01  3.95  3.98  3.87   3.87  0.371163    6000   AACG
2014-12-02  3.72  3.72  3.72   3.72  0.356777     300   AACG
2014-12-03  3.70  3.85  3.51   3.51  0.336637   12800   AACG
2014-12-04  3.51  3.61  3.26   3.30  0.316496   22100   AACG
2014-12-05  3.29  3.45  3.11   3.11  0.298274   15400   AACG
...          ...   ...   ...    ...       ...     ...    ...
2024-11-22  1.00  1.02  0.98   1.02  1.020000   51000   AACG
2024-11-25  1.00  1.02  0.97   1.02  1.020000   56400   AACG
2024-11-26  1.02  1.02  0.98   1.02  1.020000   15000   AACG
2024-11-27  1.00  1.01  0.94   1.00  1.000000   94100   AACG
2024-11-29  0.96  1.01  0.96   1.00  1.000000   24400   AACG

[2517 rows x 7 columns]


Dictionary of volumes per charts:

In [72]:
dict_volume={}
for ticker in dict_data_tickers:
    dict_volume[ticker]=dict_data_tickers[ticker]["volume"].sum()
volDF=pd.DataFrame.from_dict(dict_volume,orient="index",columns=["Volume"])
print(volDF)

            Volume
AACG     740123200
AADI     237083058
AADR      25517100
AAL    61030886200
AAME     188959800
AAOI    2942353800
AAON     769796100
AAPB      37760700
AAPD     130427000
AAPL  297906658200
AAPU     178352300
AAXJ    2735189200
ABAT     408914717
ABCL    1898549200
ABCS        905900


In [76]:
print(volDF.index)

Index(['AACG', 'AADI', 'AADR', 'AAL', 'AAME', 'AAOI', 'AAON', 'AAPB', 'AAPD',
       'AAPL', 'AAPU', 'AAXJ', 'ABAT', 'ABCL', 'ABCS'],
      dtype='object')


Highest and lowest number of volume and what chart it's from:

In [83]:
print(f"The stock chart with the biggest volume is {volDF[volDF["Volume"].max()].index} with {volDF["Volume"].max()}.")
print(f"The stock chart with the lowest volume is {minL} with {minV}.")

KeyError: 297906658200

2. Pre-processing

3. Analysis and visualizations